<a href="https://colab.research.google.com/github/mmistroni/jupyter/blob/master/TechnicalIndicatorsNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-datareader
import requests
import pandas as pd
from datetime import date
import logging
from pandas.tseries.offsets import BDay
import requests
import numpy as np
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date



In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

<h3> Useful functions to be translated to GCP </h3>

In [ ]:
from google.colab import auth
auth.authenticate_user()




In [ ]:
import pandas as pd
!mkdir -p data
!gsutil cp gs://datascience-bucket-mm/All_Indicators.txt data
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv data
!ls data

!pwd

In [ ]:
sct = pd.read_csv('data/all_sectors.csv')
sct.head(5)

sct['Sector'].unique()

<h3> Attempting to predict prices using Technica indicators </h3>

In [ ]:
# Getting Tickers..

!gsutil cp gs://datascience-bucket-mm/all_sectors.csv data
sct = pd.read_csv('data/all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)
sct.head(2)
reit_tickers = sct[sct['sector'] == 'Real Estate']
tickers = reit_tickers['ticker'].values
tickers


In [ ]:
def get_prices(ticker, start_date, end_date):
  try:
    print('Fetching {} from {} to {}'.format(ticker, start_date, end_date))
    return dr.get_data_yahoo(ticker, start_date, end_date)
  except Exception as e:
    print('could not fetch {}'.format(ticker))

<h3> Simple Moving Avreage </h3>

In [ ]:
def map_to_dict(input_df):
  dd = input_df.to_dict()
  return dict(    (k.replace(' ', ''), [vl for vl in v.values()][0]) for k, v in dd.items())

start = get_prices('AMZN', date.today(), date.today())
map_to_dict(start)
#start.to_dict()

In [ ]:
def add_moving_averages(prices):
  print('Adding moving averages....')
  prices['SMA_10'] = (sum(prices.close, 10))/10
  prices['SMA_20'] = (sum(prices.close, 20))/20
  prices['SMA_50'] = (sum(prices.close, 50))/50
  prices['SMA_100'] = (sum(prices.close, 100))/100
  prices['SMA_200'] = (sum(prices.close, 200))/200
  return prices

def add_emas(prices):
  print('Adding EMA...')
  prices['ema_10'] = prices.close.ewm(span=10).mean().fillna(0.0)
  prices['ema_20'] = prices.close.ewm(span=20).mean().fillna(0.0)
  prices['ema_50'] = prices.close.ewm(span=50).mean().fillna(0.0)
  prices['ema_100'] = prices.close.ewm(span=100).mean().fillna(0.0)
  prices['ema_200'] = prices.close.ewm(span=200).mean().fillna(0.0)
  return prices

def add_true_range(prices, timeperiod=14):
  print('Adding ATR for tperiod:{}'.format(timeperiod))
  print(prices.dtypes)
  try:
    prices['ATR'] = talib.ATR(prices.high.values,
                              prices.low.values,
                              prices.close.values,
                              timeperiod=timeperiod)
  except Exception as e:
    print('could not calculate atr')
    prices['ATR'] = None
  return prices

def add_adx(prices, timeperiod=14):
  print('Adding ADX for tperiod:{} and cols:{}'.format(timeperiod, prices.columns))

  prices['ADX'] = talib.ADX(prices.high,prices.low, prices.close, timeperiod=timeperiod)
  return prices

def add_cci(prices):
  print('Adding CCI ')

  tp = (prices['high'] + prices['low'] + prices['close']) / 3
  ma = tp / 20
  md = (tp - ma) / 20
  prices['CCI'] = (tp-ma) / (0.015 * md)
  return prices

def add_roc(prices, nshift=12):
  print('Addig rate of change for {}'.format(nshift))
  prices['ROC'] = ((prices['close'] - prices['close'].shift(nshift)) / 
                  (prices['close'].shift(nshift))) * 100
  return prices

def add_rsi(prices, timeperiod=14):
  print('Adding RSI for tp:{}'.format(timeperiod))
  try:
    prices['rsi'] = talib.RSI(prices.close.values, timeperiod=timeperiod)
  except Exception as  e:
    print('couldd no tcalculate rsi')
    prices['rsi'] = None
  return prices

def add_williams(prices, timeperiod=14):
  print('Adding williams r for tp:{}'.format(timeperiod))
  try:
    prices['Williams % R'] = talib.WILLR(prices.high.values, prices.low.values,
                                     prices.close.values, timeperiod)
  except Exception :
    print('could not calculate williams')
    prices['Williams % R'] = None
  return prices

def add_stochastic_k(prices):
  print('Adding Stochastic K ')
  prices['SO%K'] = ((prices['close'] - prices['low']) / (prices['high'] - prices['low']))
  return prices


In [ ]:
def add_technical_indocators(prices):
  sma = add_moving_averages(prices)
  emas = add_emas(sma)
  print('before atx we have:{}'.format(emas.columns))
  atr = add_true_range(emas)
  
  atx = add_adx(atr)
  cci = add_cci(atx)
  roc = add_roc(cci)
  rsi = add_rsi(roc)
  williams = add_williams(rsi)
  stochastic = add_stochastic_k(williams)
  print('Current Shape:{}'.format(prices.shape))
  print('Dropping null')
  not_null = prices.dropna()
  print('Cleaned uup has :{}'.format(not_null.shape))
  return not_null


In [ ]:
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv data
sct = pd.read_csv('data/all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)
sct.head(2)


<h3> Fetching Prices for Sector (Moved to StockNews Notebook) </h3>

1.   List item
2.   List item



In [ ]:
def get_historical_prices_for_ticker(ticker):
  print('Fetching data for:{}'.format(ticker))
  end_date = date.today()
  start_date = end_date - BDay(600)
  return get_prices(ticker, start_date, end_date)


In [ ]:
#reit_tickers = sct[sct['sector'] == 'Real Estate']
def create_historical_datasource(ticker):
  print('Creating historical datasource for {}'.format(ticker))
  tickers = [ticker]#'reit_tickers['ticker'].values
  all_data= []
  for t in tickers:
    res = get_historical_prices_for_ticker(t)
    if res is not None:
      res['ticker'] = ticker
      renamed = res.rename(columns = {'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume', 'Open':'open','Adj Close':'adj_close'}) 
      enhanced = add_technical_indocators(renamed)
      all_data.append(enhanced)
      all_reit = pd.concat( [p for p in all_data if p is not None], axis=0)#
      all_reit['result'] = (all_reit['close'].shift(-20)) / all_reit['close']
      return all_reit



In [ ]:
ccyclical = sct[sct['sector'] == 'Consumer Cyclical'].ticker.values
full_dsets = [create_historical_datasource(t) for t in ccyclical]

In [ ]:
tech_cyclical = pd.concat([d for d in full_dsets if d  is not None])

In [ ]:
tech_cyclical.shape

In [ ]:
ds = tech_cyclical.to_csv('tech_cyclical.csv')
!gsutil cp tech_cyclical.csv gs://mm_dataflow_bucket/inputs/

<h3> Restoring data </h3>

In [ ]:
!gsutil cp  gs://mm_dataflow_bucket/inputs/tech_cyclical.csv .
df = pd.read_csv('tech_cyclical.csv')

In [ ]:
print('All:{}'.format(df.shape))
print('No na:{}'.format(df.dropna().shape))
print(df[df['result'] > 1.2].shape)
df.columns

In [ ]:
print('Current hsp:{}'.format(df.shape))
print('Dropped:{}'.format(df.dropna().shape))
samples = df.tail(10)
st = df.tail(5)
st['next']= st['Date'].shift(-2)

In [ ]:
st

In [ ]:
all_data = create_historical_datasource('^DJI')
all_data['pred_price'] = all_data.close.shift(-7) > all_data.close * 1.05
all_data.to_csv('dji_last_5_years_technical_indicators_7day_increase_5pcrise.csv')
!gsutil cp dji_last_5_years_technical_indicators_7day_increase_5pcrise.csv gs://mm_dataflow_bucket/inputs/

In [ ]:
all_data['pred_price'] = all_data.close.shift(-20) > all_data.close * 1.05
all_data.to_csv('dji_last_5_years_technical_indicators_20day_increase_5pcrise.csv')
!gsutil cp dji_last_5_years_technical_indicators_20day_increase_5pcrise.csv gs://mm_dataflow_bucket/inputs/

In [ ]:
all_data[all_data['pred_price'] == True].shape
#all_reit.to_csv('dji_last_5_years_technical_indicators_7day_increase_10pcrise.csv')
#!gsutil cp dji_last_5_years_technical_indicators_7day_increase_5pcrise.csv gs://mm_dataflow_bucket/inputs/

<h3>Attepmting to find performance for sectors </h3>

<h3> Fetching all sectors </h3>

In [ ]:
!mkdir -p data
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv data
sct = pd.read_csv('data/all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)
print(sct.shape)




In [ ]:
def get_historical_price(ticker, busdays):
  print('Find price for last {} days'.format(busdays))
  end_date = date.today()
  start_date = end_date - BDay(busdays)
  historical_df =  get_prices(ticker, start_date, end_date)
  start = historical_df[ticker].values[0]
  end = historical_df[ticker].values[-1]
  data = {}
  data['ticker'] = ticker
  data['start'] = start
  data['end'] = end
  data['performance'] 
  




def get_prices_for_sector(all_sector_df, sector):
  print('Find ticker for {}'.format(sector))
  sector_df = all_sector_df[all_sector_df['sector'] == sector]
  print('For {} we got shape {}'.format(sector, sector_df.shape))





In [ ]:
df = get_historical_price('AAPL', 20)
df.info()


In [ ]:
def company_stats(ticker):
  base_url =  'https://financialmodelingprep.com/api/v3/company/profile/{ticker}'.format(ticker=ticker)
  try:
    data =  requests.get(base_url).json()['profile']
    pdict = dict(price=data['price'], range=data['range'],
                beta=data['beta'], industry=data['industry'], TICKER=ticker)
    ratings = requests.get('https://financialmodelingprep.com/api/v3/company/rating/{}'.format(ticker)).json()
    pdict['rating'] = ratings.get("rating")['recommendation'] if ratings.get("rating") else 'N/A'

    metrics = requests.get('https://financialmodelingprep.com/api/v3/company/discounted-cash-flow/{}'.format(ticker)).json()
    pdict['dcf'] = metrics.get('dcf') or 'N/A'
  except Exception as  e:
    pdict = dict(price='N/A', range='N/A',
                beta='N/A', industry='N/A', TICKER=ticker)
    

  return pd.DataFrame([pdict])





In [ ]:
df = pd.read_csv('data/edgar_quarterly_run_withindustry.csv-00000-of-00001')

In [ ]:
df

In [ ]:
import requests
from datetime import date
def get_company_stats(tpl):
    name, ticker, count = tpl
    base_url = 'https://us-central1-datascience-projects.cloudfunctions.net/company-stats/{}'.format(ticker)
    res_dict = requests.get(base_url).json()

    mapped = dict((k.upper(),v) for k,v in res_dict.items())
    upd = dict(COB=date.today().strftime('%Y-%m-%d'), CUSIP=name, TICKER=ticker,COUNT=count)
    mapped.update(upd)

    return(mapped)


In [ ]:
get_company_stats(('NAME', 'BILI', 100))

In [ ]:
import requests
requests.get('https://us-central1-datascience-projects.cloudfunctions.net/future_earnings/AAPL')

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!mkdir -p data
!gsutil cp gs://mm_dataflow_bucket/results-20200511-213128.csv data


In [ ]:
def compute_moving_averages(prices, day):
  prices['moving_avg_{}'.format(day)] = prices['adjClose'].rolling(window=day).mean() 
  return prices

def compute_simple_moving_average(prices):
  prices['simple_moving_average'] = prices['adjClose'].mean()
  return prices

def compute_exp_moving_average(prices, day):
  prices['exp_ma_{}'.format(day)] = prices.adjClose.ewm(span=day, adjust=False).mean()
  return prices

def compute_exp_moving_average_pandas(prices, day):
  prices['exp_ma_{}'.format(day)] = prices.adjClose.ewm(span=day).mean()
  return prices

def compute_macd(prices):
  prices['ema_26'] = prices.adjClose.ewm(span=26).mean()
  prices['ema_12'] = prices.adjClose.ewm(span=12).mean()
  prices['macd'] = (prices['ema_12'] - prices['ema_26'])
  return prices

def compute_true_range(prices):
  prices['tr_1'] = prices['high'] - prices['low']
  prices['tr_2'] = prices['high']- prices['adjClose'].shift(1)
  prices['tr_3'] = prices['low'] - prices['adjClose'].shift(1)
  prices['true_range'] = prices[['tr_1','tr_2','tr_3']].max(axis=1)
  return prices.drop(['tr_1', 'tr_2', 'tr_3'], axis=1)

def compute_atr(prices):
  with_true_range = compute_true_range(prices)
  with_true_range['atr'] = prices['true_range'].rolling(window=14).mean()
  return with_true_range 



<h3> Creating functions for technical indicators </h3>
Technical indicator to calculate(see sample https://medium.com/analytics-vidhya/stock-trend-prediction-with-technical-indicators-feature-engineering-and-python-code-1fa54d5806ba):
<ul>
   <li> Simple Moving Average </li>
   <li> Exponential Moving Average(10,20,50,100,200) </li><
   <li>ATR (typically derived from the 14-day moving average of a series of true range indicators) </li>
   <li> Average Directional Index ( It is a combination of the negative and positive directional movements indicators computed over a period of n past days corresponding to the input window length (typically 14 days))</li>
   <li>Commodity Channel Index (CCI)</li>
   <li> Rate of Change  </li>
   <li> Relative strength Index </li>
   <li> William % R </li>
   <li> Stochastic %K </li>
</ul>   



In [ ]:
sample_dt = open('data/sentex_technical.txt', 'r').readlines()
mapped = list(map(lambda ln: ln.split(','), sample_dt))
#split_data = sample_dt.split('\n')
#date, closep, highp,lowp, openp, volume = np.loadtxt(split_data,
#                                                     delimiter=',')
mapped[0]

def TR(d,c,h,l,o,yc):
  ''' Calculate Directional Movement 
    :param d: date
    :param c: today close
    :param h: today high
    :param l: today low
    :param o: today open
    :param yc: yesterdya close
   '''
  z = h - 1
  y = abs(h - yc)
  z = abs(l - yc)
  if y <= x >= z:
    TR = x
  elif x <= y >=z:
    TR = y
  elif x <= z >= y:
    TR = z
    return d, Tr

def DM(d,o,h,l,c,yo,yh,yl,yc):
  ''' Calculate Directional Movement 
    :param d: date
    :param o: today open
    :param h: today high
    :param l: today low
    :param c: today close
    :parma yo: yesterday open
    :param yh: ysterday high
    :param yl: yesterday low
    :param yc: yesterdya close
  
  '''
  moveUp = h - yh
  moveDown = yl - l
  if 0 < moveUp > moveDown:
    PDM = moveUp
  else:
    PDM = 0

  if 0 < moveDown > moveUP:
    NDM = moveDown
  else:
    NDM = 0

  return d, PDM, NDM


def ADX():
  PDI, NDI  = calcDI

def ATR(prices):
  '''
     Calculate Averge True Range
  '''
  prices['ATR'] = prices['TR'].mean()
  return prices
  



In [ ]:
def get_historical_prices(ticker, start_dt, end_dt):
  start_dt_str = start_dt.strftime('%Y-%m-%d')
  end_dt_str = end_dt.strftime('%Y-%m-%d')
  base_request = 'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={from_dt}&to={to_dt}'.format(ticker=ticker,
                                                                                                                            from_dt=start_dt_str,
                                                                                                                            to_dt=end_dt_str)
  print('Calling URL:{}'.format(base_request))
  all_dt =  requests.get(base_request).json().get('historical') or {}
  df =  pd.DataFrame(all_dt)[['adjClose', 'date', 'high', 'low']]
  df.set_index('date', inplace=True)
  return df

def get_today_and_yesterday(ticker, start_dt):
  today = start_dt.strftime('%Y-%m-%d')
  yday = (start_dt - BDay(1)).strftime('%Y-%m-%d')
  base_request = 'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={from_dt}&to={to_dt}'.format(ticker=ticker,
                                                                                                                            from_dt=yday,
                                                                                                                            to_dt=today)
  all_dt =  requests.get(base_request).json().get('historical') or {}
  y = all_dt[0]
  t = all_dt[1]  # yo,yh,yl,yc
  y_dict = dict(y_c=y['close'], y_h=y['high'],
             y_low=y['low'], y_o=y['open'])
  t_dict = dict(date=t['date'], t_c=t['close'], t_h=t['high'],
                t_l=t['close'], t_o=t['open'],
                ticker=ticker)
  
  y_dict.update(t_dict)
  return pd.DataFrame([y_dict])


In [ ]:
get_today_and_yesterday('AMZN', date(2019,5,6))

In [ ]:
end = date.today()
start = end - BDay(300)
amzn = get_historical_prices('AMZN', start, end)
amzn.head(10)

In [ ]:
with_50 = compute_moving_averages(amzn, 50)
with_200 = compute_moving_averages(with_50, 200)
with_ema_12 = compute_exp_moving_average(with_200, 12)
with_ema_26 = compute_exp_moving_average(with_ema_12, 26)
with_macd = compute_macd(with_ema_26)
with_atr = compute_true_range(with_macd)
#with_macd.columns


In [ ]:
prices = with_macd
prices['true_range'] = max([(prices['high'] - prices['low']), abs(prices['high']- prices['adjClose'].shift(1)), abs(prices['low'] - prices['adjClose'].shift(1))])

In [ ]:
amzn['moving_avg_50'] = amzn.adjClose.ewm(span=50, adjust=False).mean()
amzn['moving_avg_200'] = amzn.adjClose.ewm(span=200, adjust=False).mean()

#compute_moving_averages(amzn, 30)

In [ ]:
amzn['crossover'] = amzn['moving_avg_50'] > amzn['moving_avg_200']   
amzn['lookahead'] = amzn['adjClose'].shift(-5)
amzn[ (amzn['crossover'] == True) & (amzn['lookahead'] > amzn['adjClose'])].shape


In [ ]:
df = pd.read_csv('data/results-20200511-213128.csv')
df.shape

In [ ]:
def  get_price_targets(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/price-target?token={token}'.format(symbol=ticker, )
  try:
    res = requests.get(base_url).json()
    res['TICKER'] = ticker
  except Exception as e:
    res = {'TICKER': ticker,
          'currency': 'USD',
          'numberOfAnalysts': 0,
          'priceTargetAverage': 0,
          'priceTargetHigh': 0,
          'priceTargetLow': 0,
          'symbol': ticker,
          'updatedDate': 'N/A'}
  return pd.DataFrame([res])


In [ ]:
for ticker in ['ATVI','TWTR','ROK','DPZ']:
  get_price_targets(ticker)

In [ ]:
print('current sjhape:{}'.format(df.shape))
filtered_df = df[df['COUNT'] > 100]
print('filtered sjhape:{}'.format(filtered_df.shape))

all_tickers = filtered_df['TICKER'].values.tolist()


In [ ]:
dfs = pd.concat([company_stats(ticker) for ticker in all_tickers])
#estimates = pd.concat([get_price_targets(ticker) for ticker in all_tickers])

In [ ]:
estimates = pd.concat([get_price_targets(ticker) for ticker in all_tickers])
print('dfs:{}, est:{}'.format(filtered_df.shape, estimates.shape))
dfs.columns

In [ ]:
merged = pd.merge(dfs, estimates, on='TICKER')
print(merged.shape)

In [ ]:
all_dt = pd.merge(dfs, df, on='TICKER', how='inner')
all_dt.shape

In [ ]:
from datetime import date
def get_price_drop(ticker):
  base_url = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?from=2020-03-01&to={}'.format(ticker, date.today().strftime('%Y-%m-%d'))
  data = requests.get(base_url).json()
  if data.get('historical'):
    first = data['historical'][0]['close']
    last = data['historical'][-1]['close']
    return pd.DataFrame([dict(TICKER=ticker, Diff_from_March=(last - first))])
  else:
    return pd.DataFrame([dict(TICKER=ticker, Diff_from_march=0)])


In [ ]:
pchanges = pd.concat([get_price_drop(ticker)  for ticker in all_tickers])
df_changes = pd.merge(all_dt,pchanges, on='TICKER')


In [ ]:
dfs.to_csv('data/institutional_hol._14may.csv')
!gsutil cp data/institutional_hol._14may.csv gs://mm_dataflow_bucket/outputs

<h3> Attempting to find cyclical stock </h3>

In [ ]:
def get_historical_prices_for_ticker_cyclical(ticker):
  print('Fetching data for:{}'.format(ticker))
  end_date = date.today()
  start_date = end_date - BDay(240)
  return get_prices(ticker, start_date, end_date)


In [ ]:
df = get_historical_prices_for_ticker_cyclical('SLB')
mx = df.Close.max()
mn = df.Close.min()

In [ ]:
df['near_max'] = df.Close > mx * .95
df['near_min'] = df.Close < mn * 1.05


print(df[df['near_max'] == True].shape)
print(df[df['near_min'] == True].shape)